<a href="https://colab.research.google.com/github/JoungMinJu/machine_learning/blob/main/5%EC%9E%A5/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모델을 많이 만들어서 테스트 세트로 평가하면 테스트 세트에만 잘 맞는 모델이 만들어질 수 있다. 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 테스트 세트를 사용하지 말아야함. 모델 만들고 마지막에 딱 한 번만 사용하는ㄱ ㅓㅅ이 좋다.


그래서 훈련세트를 또 나눠서 검증세트로 만든다. 

In [1]:
import pandas as pd
wine=pd.read_csv('https://bit.ly/wine-date')

In [2]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target=train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
#train을 훈련세트 sub와 검증세트 val으로 만든다.
sub_input, val_input, sub_target, val_target=train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

#과대적합되어있다.

0.9971133028626413
0.864423076923077


교차검증 : 검증세트를 뗴어내어 평가하는 과정을 여러 번 반복. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 3-폴드 교차검즈이다. 훈련세트를 세 부분으로 나눠서 교차검증을 수행하는 것!! 통칭 k-폴드 교차검증이라고 한다.


In [8]:
#보통은 5폴드나 10폴드를 많이 사용한다. 이렇게 하면 데이터의 8~90%까지 훈련에 사용하 ㄹ수 있따. 검증세트가 줄어들지만 각 폴드에서 계산한 검증점수를 평균하기 떄문에 안정된 점수로 생각할 수 있다
#사이킷런에는 cross-validate9)라는 교차검증함수가 있다. 사용법은 간단한데 먼저 평가할 모델 객체를 첫번쨰 매개변수로 전달한다. 그 다음 직접 검증셑트를 떼어내지 않고 훈련세트 전체를 cross_validate함수에 전달한다.

from sklearn.model_selection import cross_validate
scores=cross_validate(dt, train_input, train_target)
print(scores)
#기본적으로 5교차검증


#이는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미. 
#교차검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다. test_score은 검증폴드의 점수임!

{'fit_time': array([0.01010776, 0.00726771, 0.00762987, 0.00731897, 0.00701714]), 'score_time': array([0.00085807, 0.00065756, 0.00068355, 0.00066185, 0.00059843]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [11]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
#교차 검증을 수행하며 ㄴ입력한 모델에서 어등ㄹ 수 있는 최상의 검증 점수를 가늠해 볼 수 있다.
#주의할 점은 corss_validate()는 훈련세트를 섞어 폴드를 나누지 않는다. 우리는 train_test_split()으로 이미 전체 데이터 섞고 훈련세트를 준비해서 따로 섞을 필요는 없지만,,
#교차검증할 때 훈련세트를 섞으려면 분할기를 지정해야한다.    
#기본적으로 회귀모델일 경우KFold 분할기 +++ 분류모델일 경우 타깃클래스를 골고루 나누기 위해 StratifiedKFold를 사용

In [14]:
from sklearn.model_selection import StratifiedKFold
scores=cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#KFold클래스도 동일한 방식으로 사용할 수 있다. 
#교차검증에 대해 이해했으니 결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보겠다. 

#테스트세틀르 사ㅛㅇ하지 않고 교차검증을 통해서 좋은 모델을 고르면 된다.

하이퍼파라미터 튜닝


우선 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련하고
검증세트의 점수나 교차검증을 통해서 매개변수를 조금씩 바꿔보기


이때!!!!!!!!!!!!!!!!!여러개의 매개변수의 최적의 조함을 찾깅 ㅟ해 그리드서치가존재한다.
얘는 하이퍼파라미터 탐색과 교차검증을 한 번에 수행한다. 별도로 corss_validate()함수를 호출할 필요가 없음. 기본 매개변수를 사용한 결정트리모델에서 min_inpurity_decrease매개변수의 최적값을 찾아보는 것이 아래의 실습

In [15]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}


In [16]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
#탐색대상모델과 탐색할 값을 전달했다.
#girdsearch의 cv 의 기본 매개변수는 5이다. 그러니까 min_impurity_decrease에 전달해주는 params의 요소 5개마다!!!!! 5폴드 교차검증을 수행하는것
#그러면 총 5*5 25개의 모델을 검증하게 된다. 많은 모델을 훈련하기 떄문에 n-_jobs에서 병렬실행에 사용할 CPU 코어 수를 지정하는 것이 좋다. 기본값은 1이고 -1은 시스템에 있는 모든 코어를 사용한다.

gs.fit(train_input, train_target)


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [ ]:
#교차검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련세트로 모델을 다시 만들어야했음.
#근데 그리드 서치는 검증점수가 가장 높으 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련한다.
#이 모델은 gs객체의 best_estimator속성에 저장되어있다!!! 이 모델을 일반 결정트리처럼 똑같이 사용할 수 있음

In [18]:
dt=gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [19]:
#최적의 매개변수는 여기에 저장되어있습니다
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [20]:
#각 매개변수에서 수행한 교차검증의 평균점수는 cv_results_속성의 mean_test_score키에 저장되어있따.
#5번의 교차검증으로 얻은 점수는

print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [21]:
#첫번째 값이 가장 큰 것을 확인할 수 있음. 수동으로 고르는 것 보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
#그리고 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.
# 이 값이 최상의 검증 점수를 만든 매개변수 조합이다. 
best_index= np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


즉
1. 탐색할 매개변수지정

2.훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증점수가 나오는 매개변수 조합을 찾음 이 조합은 그리드서치 객체에 저장된

3.그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련세트가 아니라) 전체 훈련세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드서치에 저장

In [22]:
params={'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}


#이 매개변수로 수행할 교차 검증 횟수는 9*15*10 = 1,350개
#기본 5폴드 교차검증을 수행하므로 만들어지는 모델은 총 6750개

In [23]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [25]:
print(gs.best_params_)
#최상의 교차검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [26]:
#최적의 모델은 이미 전체 훈련세트(train_input, train_target)으로 훈련되어 best_estimator_속성에 저장되어있음
#이를 최종 모델로 결정하고 테스트 세트의 성능확인

dt=gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616
